In [9]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score


In [11]:
ruta = "/content/Solicitantes de Credito.xlsx"
data = pd.read_excel(ruta)

data.head()


,Unnamed: 0,Edad,Sexo,Estado civil,Hijos,Perscargo,Estrato,Nivel de estudios,Tipo de contrato,Ingresos,...,Plazo,Cuota (COP),ConoCliente,Garantia,RespPat,MoraCome,PreApr,Prob.Default (PD),Score,LGD
0,1,38,NR,Casado,1,0,3,Bachiller,Ninguno,5424576,...,48,82691.506204,Buenas Referencias,Sin Codeudor,Sin Patrimonio,Con Experiencia sin Mora,1.0,0.041659,582.478943,0.063702
1,2,51,NR,Separado,6,1,4,Técnico,Ninguno,1144064,...,36,22781.212598,Buenas Referencias,Sin Codeudor,Prop.sin Hipoteca,Con Experiencia sin Mora,1.0,0.234951,479.839966,0.465636
2,3,37,Mujer,Casado,2,0,3,Tecnólogo,Indefinido,2069853,...,36,46294.596244,Buenas Referencias,Con Codeudor(es),Sin Patrimonio,Con Experiencia sin Mora,0.0,0.170330,417.653107,0.290071
3,4,29,NR,Separado,1,0,2,Tecnólogo,Ninguno,1893080,...,24,69564.950015,Buenas Referencias,Con Codeudor(es),Sin Patrimonio,Con Experiencia sin Mora,1.0,0.201659,316.791107,0.278389
4,5,42,Mujer,Casado,3,4,2,Especializacion,Ninguno,3000367,...,24,112478.307883,Buenas Referencias,Con Codeudor(es),Sin Patrimonio,Con Experiencia sin Mora,1.0,0.140929,441.809570,0.203359


In [12]:
data.info()
data.isnull().sum()
data = data.dropna()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8378 entries, 0 to 8377
Data columns (total 35 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               8378 non-null   int64  
 1   Edad                     8378 non-null   int64  
 2   Sexo                     8378 non-null   object 
 3   Estado civil             8378 non-null   object 
 4   Hijos                    8378 non-null   int64  
 5   Perscargo                8378 non-null   int64  
 6   Estrato                  8378 non-null   int64  
 7   Nivel de estudios        8378 non-null   object 
 8   Tipo de contrato         8378 non-null   object 
 9   Ingresos                 8378 non-null   int64  
 10  Egresos                  8378 non-null   int64  
 11  Tipo de actividad        8378 non-null   object 
 12  Tipo de vivienda         8378 non-null   object 
 13  Total Activos            8378 non-null   float64
 14  Total pasivos           

In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.naive_bayes import GaussianNB

In [37]:

data.columns = data.columns.str.strip()
df_clean = data.dropna(subset=['PreApr'])
columnas_modelo = [
    'Edad', 'Hijos', 'Perscargo', 'Estrato', 'Ingresos', 'Egresos',
    'Total Activos', 'Total pasivos', 'Salario', 'Gasto familiar',
    'Gasto obligatorio', 'Gasto arriendo', 'Otros gastos',
    'Monto (EAD)', 'Plazo', 'Cuota (COP)'
]

XD = df_clean[columnas_modelo]
yd = df_clean['PreApr']

XDArray = np.array(XD)
ydArray = np.array(yd)


In [38]:

X_train, X_test, y_train, y_test = train_test_split(
    XDArray, ydArray, test_size=0.2, random_state=42
)

modelo_nb = GaussianNB()

modelo_nb.fit(X_train, y_train)


GaussianNB()

In [46]:

from sklearn.metrics import confusion_matrix

yd_pred = modelo_nb.predict(X_test)
cm = confusion_matrix(y_test, yd_pred)


VN = cm[0, 0]
FP = cm[0, 1]
FN = cm[1, 0]
VP = cm[1, 1]


exactitud = (VP + VN) / (VP + VN + FP + FN)
error = (FP + FN) / cm.sum()
precision = VP / (FP + VP)
sensibilidad = VP / (VP + FN)
especificidad = VN / (FP + VN)


print(" MATRIZ DE CONFUSIÓN ")
tabla_cm = pd.crosstab(y_test, yd_pred, rownames=['Real'], colnames=['Predicción'])
print(tabla_cm)


print(f"La Exactitud es: {exactitud:.4f}")
print(f"El Error de Clasificación es: {error:.4f}")
print(f"La Precisión es: {precision:.4f}")
print(f"La Sensibilidad es: {sensibilidad:.4f}")
print(f"La Especificidad es: {especificidad:.4f}")

print("\nInterpretación:")
print(f"El modelo tiene una capacidad del {sensibilidad*100:.2f}% para detectar correctamente los créditos pre-aprobados.")

 MATRIZ DE CONFUSIÓN 
Predicción  0.0  1.0
Real                
0.0         497  108
1.0         202  362
La Exactitud es: 0.7348
El Error de Clasificación es: 0.2652
La Precisión es: 0.7702
La Sensibilidad es: 0.6418
La Especificidad es: 0.8215

Interpretación:
El modelo tiene una capacidad del 64.18% para detectar correctamente los créditos pre-aprobados.


In [48]:
print(f"El modelo clasifica correctamente el {exactitud*100:.2f}% de los casos.")
print(f"Tiene una capacidad del {sensibilidad*100:.2f}% para identificar créditos pre-aprobados.")

El modelo clasifica correctamente el 73.48% de los casos.
Tiene una capacidad del 64.18% para identificar créditos pre-aprobados.


En conclusión, aunque el modelo del reto propuesto  hizo un buen trabajo, no fue tan "Exacto" como el que vimos en la clase. Nuestra precisión general quedó en un 73%, mientras que en la clase logramos un 80%.

¿Por qué pasó esto?
Los datos de la clase eran casi perfectos. En cambio, los del reto son de solicitudes de crédito de personas que cuentan con perfiles  muy variados. Esto hace que al modelo le cueste más encontrar un patrón claro.

El modelo que usamos supone que cada dato, como el sueldo y los gastos, no tienen nada que ver entre sí. Pero en la realidad, el que gana más suele gastar más. Esa relación "confunde" un poco al modelo y por eso se puede decir que fallo un poco mas.

El modelo, tambien se volvió un poco mas estricto. Nuestra Sensibilidad bajó al 64%, lo que significa que el modelo prefirió decirle que "no" a varias personas que en realidad sí eran buenas pagadoras, solo para no arriesgarse.